# 한글 Parent-Child Chunk 를 OpenSearch 에 저장

---

---

# 1. Bedrock Client 생성

In [2]:
! pip list | grep langchain # 0.0.312
! pip list | grep opensearch # 2.3.2
! pip list | grep pypdf

langchain                            0.0.347
langchain-core                       0.0.11

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
opensearch-py                        2.4.2

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
pypdf                                3.17.1

[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [3]:
%load_ext autoreload
%autoreload 2

import sys, os
# module_path = "../../../utils"
# sys.path.append(os.path.abspath(module_path))
# print(os.path.abspath(module_path))

module_path = "./utils"
sys.path.append(os.path.abspath(module_path))
print(os.path.abspath(module_path))

/root/aws-ai-ml-workshop-kr/genai/aws-gen-ai-kr/20_applications/02_qa_chatbot/01_preprocess_docs/utils


In [4]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

# ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----

# os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
# os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
# os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
# os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."


boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print(colored("\n== FM lists ==", "green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


# 2. Embedding 모델 로딩

## Embedding Model 선택

In [5]:
Use_Titan_Embedding = True
Use_Cohere_English_Embedding = False

## Embedding Model 로딩

In [6]:
# We will be using the Titan Embeddings Model to generate our Embeddings.
from langchain.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock

if Use_Titan_Embedding:
    llm_emb = BedrockEmbeddings(client=boto3_bedrock, model_id = "amazon.titan-embed-text-v1")
    dimension = 1536
elif Use_Cohere_English_Embedding:
    llm_emb = BedrockEmbeddings(client=boto3_bedrock, model_id = "cohere.embed-english-v3")    
    dimension = 1024
else:
    lim_emb = None

llm_emb

BedrockEmbeddings(client=<botocore.client.BedrockRuntime object at 0x7f1f2fd8e980>, region_name=None, credentials_profile_name=None, model_id='amazon.titan-embed-text-v1', model_kwargs=None, endpoint_url=None)

# 3. Load all Json files

In [8]:
from utils.proc_docs import get_load_json, show_doc_json

In [9]:
import glob

# Specify the directory and file pattern for .txt files
folder_path = 'data/poc/preprocessed_json/*ko.json'

# List all .txt files in the specified folder
json_files = glob.glob(folder_path)
# json_files = ['data/poc/customer_EFOTA.json']

# Load each item per json file and append to a list
doc_json_list = []
for file_path in json_files:
    doc_json = get_load_json(file_path)
    doc_json_list.append(doc_json)

print("all json files: ", len(doc_json_list))    
# Flatten the list of lists into a single list
all_docs = []
for item in doc_json_list:
        all_docs.extend(item)
        
print("all items: ", len(all_docs))

all json files:  1
all items:  1732


# 4. Index 생성

## Index 이름 결정

In [10]:
index_name = "v18-genai-poc-knox-kor-parent-doc-retriever"

## Index 스키마 정의

In [11]:
index_body = {
    'settings': {
        'analysis': {'analyzer': {'my_analyzer': {'char_filter': ['html_strip'],
                                                    'tokenizer': 'nori',
                                                       'filter': [
                                                                   'nori_number',
                                                                   'lowercase',
                                                                   'trim',
                                                                   'my_nori_part_of_speech'
                                                                 ],
                                                       'type': 'custom'}},
                                   'tokenizer': {'nori': {
                                                  'decompound_mode': 'mixed',
                                                  'discard_punctuation': 'true',
                                                  'type': 'nori_tokenizer'}
                                                },
                                    "filter": {
                                          "my_nori_part_of_speech": {
                                                "type": "nori_part_of_speech",
                                                "stoptags": [
                                                      "E", "IC", "J", "MAG", "MAJ",
                                                      "MM", "SP", "SSC", "SSO", "SC",
                                                      "SE", "XPN", "XSA", "XSN", "XSV",
                                                      "UNA", "NA", "VSV"
                                                ]
                                          }
                                    }
                    },        
        'index': {
            'knn': True,
            'knn.space_type': 'cosinesimil'  # Example space type
        }
    },
    'mappings': {
        'properties': {
            'metadata': {
                'properties': {
                               'source' : {'type': 'keyword'},                    
                               'last_updated': {'type': 'date'},
                               'project': {'type': 'keyword'},
                               'seq_num': {'type': 'long'},
                               'title': {'type': 'text'},  # For full-text search
                               'url': {'type': 'text'},  # For full-text search
                            }
            },            
            'text': {
                'analyzer': 'my_analyzer',
                'search_analyzer': 'my_analyzer',
                'type': 'text'
            },
            'vector_field': {
                'type': 'knn_vector',
                'dimension': 1536  # Replace with your vector dimension
            }
        }
    }
}


# 5. LangChain OpenSearch VectorStore 생성 
## 선수 조건


## 오픈 서치 도메인 및 인증 정보 세팅

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.

In [12]:
from utils.proc_docs import get_parameter

In [13]:
import boto3
ssm = boto3.client('ssm', 'us-east-1')

opensearch_domain_endpoint = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_domain_endpoint',
)

opensearch_user_id = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_userid',
)

opensearch_user_password = get_parameter(
    boto3_clinet = ssm,
    parameter_name = 'knox_opensearch_password',
)


In [14]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

## OpenSearch Client 생성

In [15]:
from utils.opensearch import opensearch_utils

In [16]:
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 오픈 서치 인덱스 생성 
- 오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [17]:
from utils.opensearch import opensearch_utils

In [18]:

index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        index_name
    )

opensearch_utils.create_index(os_client, index_name, index_body)
index_info = os_client.indices.get(index=index_name)
print("Index is created")
pprint(index_info)

index_name=v18-genai-poc-knox-kor-parent-doc-retriever, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'v18-genai-poc-knox-kor-parent-doc-retriever'}
Index is created
{'v18-genai-poc-knox-kor-parent-doc-retriever': {'aliases': {},
                                                 'mappings': {'properties': {'metadata': {'properties': {'last_updated': {'type': 'date'},
                                                                                                         'project': {'type': 'keyword'},
                                                                                                         'seq_num': {'type': 'long'},
                                                                                                         'source': {'type': 'keyword'},
                                                                                                         'title': {'type': 'text'},
                                               

## 랭체인 인덱스 연결 오브젝트 생성

- [langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.opensearch_vector_search.OpenSearchVectorSearch.html)

In [19]:
from langchain.vectorstores import OpenSearchVectorSearch

In [20]:
vector_db = OpenSearchVectorSearch(
    index_name=index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
vector_db

# 6. Chunking JSON Doc 

## Chunk Size and Chunk Overlap Size 결정

In [21]:
parent_chunk_size = 4096
parent_chunk_overlap = 0

child_chunk_size = 1024
child_chunk_overlap = 256

opensearch_parent_key_name = "parent_id"
opensearch_family_tree_key_name = "family_tree"


In [22]:
from utils.proc_docs import create_parent_chunk, create_child_chunk

## Parent Chunking

create_parent_chunk() 아래와 같은 작업을 합니다.
- all_docs 에 있는 문서를 parent_chunk_size 만큼으로 청킹 합니다.
- Parent Chunk 에 두개의 메타 데이타를 생성 합니다.
    - family_tree: parent
    - parent_id : None

In [23]:
parent_chunk_docs = create_parent_chunk(all_docs, opensearch_parent_key_name, 
                                        opensearch_family_tree_key_name,parent_chunk_size, parent_chunk_overlap)
print(f"Number of parent_chunk_docs= {len(parent_chunk_docs)}")


Number of parent_chunk_docs= 1952


In [24]:
parent_chunk_docs[0:1]

[Document(page_content='AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': 'AR 데모 바코드', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16', 'family_tree': 'parent', 'parent_id': None})]

OpenSearch 에 parent chunk 삽입

In [25]:
%%time

parent_ids = vector_db.add_documents(
                        documents = parent_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )



CPU times: user 4.99 s, sys: 209 ms, total: 5.2 s
Wall time: 4min 28s


In [26]:
total_count_docs = opensearch_utils.get_count(os_client, index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 1952, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


삽입된 Parent Chunk 의 첫번째를 확인 합니다. family_tree, parent_id 의 값을 확인 하세요.

In [27]:
def show_opensearch_doc_info(response):
    print("opensearch document id:" , response["_id"])
    print("family_tree:" , response["_source"]["metadata"]["family_tree"])
    print("parent document id:" , response["_source"]["metadata"]["parent_id"])
    print("parent document text: \n" , response["_source"]["text"])

response = opensearch_utils.get_document(os_client, doc_id = parent_ids[0], index_name = index_name)
show_opensearch_doc_info(response)    

opensearch document id: 19d34d66-d917-456c-b3fb-ccd3e9497d00
family_tree: parent
parent document id: None
parent document text: 
 AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6


## Child Chunking

### Child Chunk 생성

아래의 create_child_chunk() 는 다음과 같은 작업을 합니다.
- parent_chunk_docs 각각에 대해서 Child Chunk 를 생성 합니다. 
- Child Chunk 에 두개의 메타 데이타를 생성 합니다.
    - family_tree: child
    - parent_id : parent 에 대한 OpenSearch document id

In [28]:
# child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
child_chunk_docs = create_child_chunk(child_chunk_size, child_chunk_overlap, parent_chunk_docs, parent_ids, 
                                      opensearch_parent_key_name, opensearch_family_tree_key_name)
print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")


Number of child_chunk_docs= 4071


### 생성된 Child 와 이에 대한 Parent 정보 확인

Child Chunk 한개에 대한 정보를 확인 합니다.

In [29]:
child_chunk_docs[0:1]

[Document(page_content='AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': 'AR 데모 바코드', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16', 'family_tree': 'child', 'parent_id': '19d34d66-d917-456c-b3fb-ccd3e9497d00'})]

Child 에 대한 Parent 정보 확인

In [30]:
parent_id = child_chunk_docs[0].metadata["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = index_name)
show_opensearch_doc_info(response)    


child's parent_id:  19d34d66-d917-456c-b3fb-ccd3e9497d00

###### Search parent in OpenSearch
opensearch document id: 19d34d66-d917-456c-b3fb-ccd3e9497d00
family_tree: parent
parent document id: None
parent document text: 
 AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6


### OpenSearch 에 Child 삽입

In [31]:
%%time

child_ids = vector_db.add_documents(
                        documents = child_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )

print("length of child_ids: ", len(child_ids))

length of child_ids:  4071
CPU times: user 9.87 s, sys: 623 ms, total: 10.5 s
Wall time: 7min 19s


In [32]:
total_count_docs = opensearch_utils.get_count(os_client, index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 6023, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [33]:
response = opensearch_utils.get_document(os_client, doc_id = child_ids[0], index_name = index_name)
show_opensearch_doc_info(response)    

opensearch document id: 253f53f8-2497-4442-b803-569f519db8b1
family_tree: child
parent document id: 19d34d66-d917-456c-b3fb-ccd3e9497d00
parent document text: 
 AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6


# 7. 검색 테스트

## Lexical 검색

In [36]:
# q = "how to add image"
q = "이미지를 어떻게 추가해"
query ={'query': 
        {'bool': {'must': 
                  [{'match': 
                    {'text': 
                     {'query': "{q}", 'minimum_should_match': '0%', 'operator': 'or'}}}], 
                  'filter': {
                    "term": {
                      "metadata.family_tree": "child"
                    }                      
                  }
                 }
        }
       }
pprint(query)

{'query': {'bool': {'filter': {'term': {'metadata.family_tree': 'child'}},
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': '{q}'}}}]}}}


In [37]:
# query = "how to add image"
# query = opensearch_utils.get_query(
#     query=query
# )

response = opensearch_utils.search_document(os_client, query, index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

# of searched docs:  10
# of display: 3
---------------------
_id in index:  be8cb695-34c6-4080-91ad-c3aa1e0e5e83
9.867163
.Q - 만료된 라이선스를 갱신하면 라이선스가 발효되고 관련 디바이스가 즉시 등록됩니까?A - iOS 및 Windows 장치의 경우 즉시 등록됩니다.Android 장치의 경우 시스템에 설정된 일정에 따라 등록하거나 장치 명령을 전송하여 등록할 수 있습니다.Q - 라이선스가 만료되면 디바이스가 등록된 순서대로 만료됩니까?A - 다양한 디바이스에서 사용 중인 라이선스가 만료되면 모든 디바이스에서 Knox Manage를 동시에 사용할 수 없게 됩니다.Q - 갱신된 라이선스를 새 디바이스 (이미 등록된 디바이스가 아닌) 에만 할당하려면 어떻게 해야 하나요?A - 라이선스 수를 늘린 후에는 먼저 기존 등록 장치의 등록을 취소한 다음 새 장치를 등록해야 합니다.그러면 갱신된 라이선스가 새 디바이스에 할당됩니다.Q - Knox Manage 콘솔에서 디바이스로 공장 초기화 명령을 보내면 라이선스가 자동으로 취소되나요?A - 공장 초기화가 성공적으로 완료되면 라이선스가 자동으로 취소됩니다.공장 초기화에 실패하더라도 라이선스는 취소되지 않으며 디바이스는 등록 취소됩니다.Q - Windows 및 크롬 OS 장치에 사용되는 라이선스는 무엇입니까?A - 윈도우 및 크롬 OS 디바이스는 Knox Manage 라이선스에 등록되어 있습니다.
{'source': 'all_processed_data_ko.json', 'seq_num': 894, 'title': '라이선스 개요', 'url': 'https://docs.samsungknox.com/admin/knox-manage/configure/licenses/licenses-overview', 'project': 'KM', 'last_updated': '2023-07-26', 'family_tree': 'child', 'pa

## 시맨틱 검색

In [38]:
vector_db.similarity_search(q, k=2)

[Document(page_content='커스텀 애니메이션 파일을 생성합니다.이미지를 업로드하여 순서대로 재생하고 원하는 이미지 방향, 디더링 및 크기를 설정하여 사용자 지정 장치 부팅 애니메이션을 만들 수 있습니다.만들고 업로드한 후에는 애니메이션을 프로필에 추가하고 장치에 할당하기 전에 애니메이션을 미리 보고 확인할 수 있습니다.애니메이션 만들기 커스텀 애니메이션을 만들려면: 1.왼쪽 탐색 메뉴에서 라이브러리를 클릭하고 미디어 탭을 클릭합니다. 2.오른쪽 상단의 미디어 추가를 클릭합니다. 3.애니메이션 생성을 클릭하여 새 애니메이션 시퀀스를 생성합니다. 4.PNG 시퀀스 업로드 화면에서 애니메이션 프레임으로 사용할 PNG 이미지를 최소 2개, 최대 99개까지 업로드합니다.최적의 디스플레이를 위해 업로드하는 PNG 파일의 크기 및 해상도가 대상 장치의 화면 크기 및 디스플레이 해상도와 동일한지 확인하십시오. 5.애니메이션에 이름을 지정하고 다음 설정을 구성합니다. 회전 - 애니메이션의 회전 각도를 선택합니다.크기 조정 - 기기에서 애니메이션을 어떻게 표시할지 선택합니다.디더링 - 파일 크기를 줄이는 데 사용되는 디더링의 양을 지정합니다.단, 설정이 높을수록 파일 크기는 줄어들지만 애니메이션 품질에 영향을 미칠 수 있습니다. 6.작업이 완료되면 애니메이션 생성을 클릭하여 애니메이션을 생성하십시오.애니메이션 업로드 기존 애니메이션을 QMG 파일로 업로드할 수도 있습니다.미디어 추가 화면에서 애니메이션 업로드를 선택하고 애니메이션 파일을 업로드합니다..QMG 형식의 파일만 지원됩니다.미디어 파일 삭제 라이브러리에서 미디어 파일을 삭제하려면: 1.왼쪽 탐색 메뉴에서 [라이브러리] 를 클릭합니다. 2.현재 프로필에서 사용 중이 아닌 미디어 파일을 선택합니다. 3.오른쪽 상단의 미디어 삭제를 클릭합니다. 4.파일 삭제를 클릭하여 파일 삭제를 확인합니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1

# 8. 검증 인덱스 생성

## Index 이름 결정

In [39]:
eval_index_name = "v17-genai-poc-knox-kor-eval-parent-doc-retriever"

## Sampling

In [40]:
import random
def get_sampling_doc(seed, ratio, docs):

    random.seed(seed)
    
    eval_docs = docs[:int(len(docs)*ratio)]
    
    return eval_docs
    
eval_docs = get_sampling_doc(seed=200, ratio=0.05, docs= all_docs)
print("eval docs: ", len(eval_docs))
eval_docs[0:2]
    
    

eval docs:  86


[Document(page_content='AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': 'AR 데모 바코드', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16'}),
 Document(page_content='동영상.이 섹션에는 Knox Capture와 관련된 제품 및 사용 방법 비디오가 포함되어 있습니다.삼성 Knox Capture 시작하기 이 비디오에서는 삼성 Knox Capture를 사용하여 모바일 디바이스를 강력한 바코드 스캐너로 변환하여 바코드 데이터를 읽고, 처리하고, 다른 애플리케이션으로 출력하는 방법을 보여 드리겠습니다.삼성 Knox Capture: Galaxy 디바이스의 엔터프라이즈급 모바일 스캔 솔루션 이 제품 소개 비디오는 Galaxy XCover Pro와 같은 견고한 삼성 디바이스를 엔터프라이즈급 바코드 스캐너로 전환하는 것이 얼마나 쉬운지 보여줍니다.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 2, 'title': '동영상', 'url': 'https://docs.samsungknox.com/admin/knox-capture/how-to-videos', 'project': 'KCAP', 'last_updated': '2023-07-26'})]

## 오픈 서치 인덱스 유무에 따라 삭제
오픈 서치에 해당 인덱스가 존재하면, 삭제 합니다. 

In [41]:
index_exists = opensearch_utils.check_if_index_exists(
    os_client,
    eval_index_name
)

if index_exists:
    opensearch_utils.delete_index(
        os_client,
        eval_index_name
    )
    
opensearch_utils.create_index(os_client, eval_index_name, index_body)
index_info = os_client.indices.get(index=eval_index_name)
print("Index is created")
pprint(index_info)    

index_name=v17-genai-poc-knox-kor-eval-parent-doc-retriever, exists=False

Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'v17-genai-poc-knox-kor-eval-parent-doc-retriever'}
Index is created
{'v17-genai-poc-knox-kor-eval-parent-doc-retriever': {'aliases': {},
                                                      'mappings': {'properties': {'metadata': {'properties': {'last_updated': {'type': 'date'},
                                                                                                              'project': {'type': 'keyword'},
                                                                                                              'seq_num': {'type': 'long'},
                                                                                                              'source': {'type': 'keyword'},
                                                                                                              'title': {'type': 'text'},
       

## 검증 인덱스 연결 오브젝트 생성

In [42]:
eval_vector_db = OpenSearchVectorSearch(
    index_name= eval_index_name,
    opensearch_url=opensearch_domain_endpoint,
    embedding_function=llm_emb,
    http_auth=http_auth, # http_auth
    is_aoss =False,
    engine="faiss",
    space_type="l2",
    bulk_size=100000,
    timeout=60    
)
eval_vector_db

## Parent Chunking

In [43]:
parent_chunk_docs = create_parent_chunk(eval_docs, opensearch_parent_key_name, 
                                        opensearch_family_tree_key_name,parent_chunk_size, 
                                        parent_chunk_overlap)
print(f"Number of parent_chunk_docs= {len(parent_chunk_docs)}")


Number of parent_chunk_docs= 87


In [44]:
parent_chunk_docs[0:1]

[Document(page_content='AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': 'AR 데모 바코드', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16', 'family_tree': 'parent', 'parent_id': None})]

In [45]:
%%time

parent_ids = eval_vector_db.add_documents(
                        documents = parent_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )



CPU times: user 222 ms, sys: 17.1 ms, total: 239 ms
Wall time: 10.6 s


In [46]:
total_count_docs = opensearch_utils.get_count(os_client, eval_index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 87, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [47]:
response = opensearch_utils.get_document(os_client, doc_id = parent_ids[0], index_name = eval_index_name)
show_opensearch_doc_info(response)    

opensearch document id: 57d41441-0336-4625-9389-eba1421acffb
family_tree: parent
parent document id: None
parent document text: 
 AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6


## Child Chunking

In [48]:
# child_chunk_docs = create_child_chunk(parent_chunk_docs[0:1], parent_ids)
child_chunk_docs = create_child_chunk(child_chunk_size, child_chunk_overlap, parent_chunk_docs, 
                                      parent_ids, 
                                      opensearch_parent_key_name, opensearch_family_tree_key_name)
print(f"Number of child_chunk_docs= {len(child_chunk_docs)}")


Number of child_chunk_docs= 152


In [49]:
child_chunk_docs[0:1]

[Document(page_content='AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 1, 'title': 'AR 데모 바코드', 'url': 'https://docs.samsungknox.com/admin/knox-capture/ar-demo-barcodes', 'project': 'KCAP', 'last_updated': '2023-10-16', 'family_tree': 'child', 'parent_id': '57d41441-0336-4625-9389-eba1421acffb'})]

In [50]:
parent_id = child_chunk_docs[0].metadata["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = eval_index_name)
show_opensearch_doc_info(response)    


child's parent_id:  57d41441-0336-4625-9389-eba1421acffb

###### Search parent in OpenSearch
opensearch document id: 57d41441-0336-4625-9389-eba1421acffb
family_tree: parent
parent document id: None
parent document text: 
 AR 데모 바코드.다음 바코드를 사용하여 녹스 캡처 AR 기능을 테스트하세요.바코드 1 바코드 2 바코드 3 바코드 4 바코드 5 바코드 6


In [51]:
%%time

child_ids = eval_vector_db.add_documents(
                        documents = child_chunk_docs, 
                        vector_field = "vector_field",
                        bulk_size = 1000000
                    )

print("length of child_ids: ", len(child_ids))

length of child_ids:  152
CPU times: user 376 ms, sys: 10.9 ms, total: 387 ms
Wall time: 16.6 s


In [52]:
total_count_docs = opensearch_utils.get_count(os_client, eval_index_name)
print("total count docs: ", total_count_docs)


total count docs:  {'count': 239, '_shards': {'total': 5, 'successful': 5, 'skipped': 0, 'failed': 0}}


In [53]:
response = opensearch_utils.get_document(os_client, doc_id = child_ids[5], index_name = eval_index_name)
show_opensearch_doc_info(response)    

opensearch document id: edf62f6a-10bf-41a0-91e2-c5f345f8a869
family_tree: child
parent document id: 6917b636-d93f-4dff-a09c-d00ae44c5495
parent document text: 
 .#How -to 주제 주요 Knox Capture 기능에 대한 자세한 설명과 자세한 설명을 보려면 다음 사용 방법 항목을 참조하십시오. 자세한 내용은 라이선스 관리 - Knox Capture 라이선스 절차에 대해 알아보고 앱에서 Knox Capture와 Knox Capture: Scandit Edition 라이선스 간에 전환하는 방법에 대해 알아보세요.스캔 프로필 - 프로필의 작동 방식, 기본 프로필의 기능, 다양한 스캔 시나리오에 맞게 새 프로필을 만드는 방법을 알아보세요.앱 및 활동 - 카메라 스캐너를 실행할 수 있는 비즈니스 앱을 정의하는 모든 스캔 프로필의 구성 요소입니다.앱과 활동의 작동 방식과 스캔 프로필에서 앱과 활동을 사용자 지정하는 방법을 알아보세요.스캔 엔진 설정 - 이 설정은 바코드 스캔 방법과 각 스캔 후 데이터에 어떤 일이 발생하는지 정의합니다.바코드 유형 설정, 스캔 모드 변경, 뷰파인더 크기 제어 등에 대해 알아보십시오.키스트로크 출력 - 스캔에 성공할 때마다 데이터를 포맷하는 방법을 정의하는 스캔 프로파일 설정입니다.스캔한 데이터에 특수 문자를 추가하거나 앞에 특수 문자를 추가하는 방법 또는 스프레드시트와 양식에서 쉽게 처리할 수 있도록 각 바코드 끝에 ASCII 탭과 Enter 키를 삽입하는 방법을 알아봅니다.테스트 모드 - 배포를 위해 구성을 내보내기 전에 스캔 프로필의 앱, 바코드 유형 및 키 입력 출력 규칙을 테스트할 수 있는 기능입니다.EMM/UEM에 구성 배포 - 다양한 스캔 프로필 내보내기 옵션과 EMM/UEM에서 내보낸 구성 파일을 사용하여 Knox Capture를 최종 사용자에게 배포하는 방법에 대해 알아봅니다.카메라 스캔 트리거 

In [54]:
parent_id = response["_source"]["metadata"]["parent_id"]
print("child's parent_id: ", parent_id)
print("\n###### Search parent in OpenSearch")
response = opensearch_utils.get_document(os_client, doc_id = parent_id, index_name = eval_index_name)
show_opensearch_doc_info(response)    


child's parent_id:  6917b636-d93f-4dff-a09c-d00ae44c5495

###### Search parent in OpenSearch
opensearch document id: 6917b636-d93f-4dff-a09c-d00ae44c5495
family_tree: parent
parent document id: None
parent document text: 
 개요.Knox Capture가 무엇이고 어떻게 작동하는지 알아보고 기본 사항부터 시작해 보세요.녹스 캡처란 무엇인가요?녹스 캡처의 기능을 설명하려면 먼저 *웨지* 스캐너가 무엇인지, 그리고 이 스캐너가 Knox Capture 솔루션과 어떤 관련이 있는지 이해하는 것부터 시작하는 것이 도움이 될 수 있습니다.기존의 하드웨어 기반 바코드 스캔 용어로 웨지 스캐너는 키보드와 컴퓨터 사이에 연결하는 물리적 장치입니다.키보드는 스캐너에 연결되고 스캐너는 컴퓨터에 연결됩니다.컴퓨터의 관점에서 보면 스캐너는 마치 추가 키보드처럼 작동합니다.컴퓨터에서 소프트웨어 프로그램 (예: 인벤토리 스프레드시트 앱) 을 실행하고 웨지 스캐너를 사용하여 바코드를 읽으면 키 입력 형태로 스캐너에서 프로그램으로 데이터가 전송됩니다.프로그램의 관점에서 보면 데이터는 마치 누군가가 키보드를 사용하여 수동으로 입력한 것처럼 보입니다.Knox Capture 작동 방식 비슷한 방식으로 Knox Capture는 소프트웨어 프로그램과 키보드 사이의 쐐기 역할을 하기도 합니다.하지만 Knox Capture는 물리적 스캔 디바이스가 아니라 호환되는 Android 스마트폰 및 태블릿용으로 설계된 모바일 솔루션입니다.기업 IT 관리자는 Google Play에서 Knox Capture 앱을 다운로드하거나 Knox Capture SDK (소프트웨어 개발 키트) 를 사용하여 자체 웨지 스캐너 앱을 맞춤 구축할 수 있습니다.Knox Capture가 디바이스에 설치되고 실행되면 애플리케이션 (예: 인벤토리 추적에 사용되는 비즈니

## 검색 테스트

In [59]:
# q = "'how to use barcode"
q = "바코드를 어떻게 사용해"
query ={'query': 
        {'bool': {'must': 
                  [{'match': 
                    {'text': 
                     {'query': f"{q}", 'minimum_should_match': '0%', 'operator': 'or'}}}], 
                  'filter': {
                    "term": {
                      "metadata.family_tree": "child"
                    }                      
                  }
                 }
        }
       }
pprint(query)

{'query': {'bool': {'filter': {'term': {'metadata.family_tree': 'child'}},
                    'must': [{'match': {'text': {'minimum_should_match': '0%',
                                                 'operator': 'or',
                                                 'query': '바코드를 어떻게 사용해'}}}]}}}


In [60]:
response = opensearch_utils.search_document(os_client, query, eval_index_name)
opensearch_utils.parse_keyword_response(response, show_size=3)

# of searched docs:  10
# of display: 3
---------------------
_id in index:  e12fde5b-d655-43cc-8eb5-bbd0eb5111d5
6.4662046
.특정 바코드 유형의 설정 메뉴를 보려면 해당 이름을 누릅니다.다음 표는 표시될 수 있는 바코드 유형 설정을 설명합니다. 설정 설명 옵션 기본 최소 길이 디코딩할 지원되는 최소 바코드 길이 (활성 기호 수) 를 결정합니다.기본적으로 활성화되지 않은 길이를 포함하거나 특정 개수의 심볼에 대한 디코딩 성능을 최적화하려면 이 속성을 변경하십시오.문자-기호 매핑은 바코드 유형에 따라 달라집니다.예를 들어 일부 바코드에는 기호 수에 추가 체크섬 문자가 포함됩니다.활성 기호 수에 대한 자세한 내용은 활성 기호 수 구성을 참조하십시오.바코드 유형에 따라 다릅니다.최대 길이: 디코딩할 지원되는 최대 바코드 길이 (활성 기호 수) 를 결정합니다.바코드 유형에 따라 다릅니다.반전 색상 인식: 어두운 배경에 밝은 잉크를 사용하여 인쇄된 반전 바코드를 인식할 수 있습니다.ON/OFF Quiet Zone 크기를 줄여 바코드 유형 사양에서 허용하는 것보다 일반적인 저소음 영역 (코드 앞뒤 공백) 보다 작은 코드를 스캔할 수 있습니다.여백 지대 사양을 위반하는 코드를 스캔할 때 문제가 발생하는 경우 이 방법을 사용하십시오.이 기능을 켜면 경우에 따라 스캔이 오탐으로 이어질 수 있다는 점에 유의하십시오.FNC1으로 이어지는 켜기/끄기 스트립은 GS1 코드를 나타내는 FNC1 선행 문자를 제거합니다.ON/OFF 직접 부품 마킹 모드 직접 부품 표시 (DPM) 데이터 매트릭스 코드를 읽을 때 스캔 성능이 향상됩니다.이 설정을 사용하면 프레임 처리 시간이 늘어난다는 점에 유의하십시오.삼성에서는 직접 부품 표시 모드를 사용할 때 성능 향상을 위해 스캔 영역을 화면의 작은 부분으로 제한할 것을 권장합니다.ON/OFF 체크섬 모드 n 바코드 체크섬은 바코드 데이터와 체크섬 알고리즘

In [61]:
eval_vector_db.similarity_search(q, k=2)

[Document(page_content='바코드 캡처가 비즈니스 앱에서 작동하지 않습니다.개요 바코드 데이터가 비즈니스 앱에 제대로 출력되지 않는 상황이 발생할 수 있습니다.비즈니스 앱에서 바코드 캡처가 작동하지 않는 이유는 무엇인가요?대부분의 바코드 캡처 문제는 키 입력 출력 및 스캔 엔진 설정의 사소한 오류에서 시작됩니다.프로필을 설정한 후 테스트 모드를 사용하여 구성을 확인해 보십시오.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 51, 'title': '바코드 캡처가 비즈니스 앱에서 작동하지 않음', 'url': 'https://docs.samsungknox.com/admin/knox-capture/kbas/barcode-capture-not-working-for-app', 'project': 'KCAP', 'last_updated': '2023-07-26', 'family_tree': 'parent', 'parent_id': None}),
 Document(page_content='바코드 캡처가 비즈니스 앱에서 작동하지 않습니다.개요 바코드 데이터가 비즈니스 앱에 제대로 출력되지 않는 상황이 발생할 수 있습니다.비즈니스 앱에서 바코드 캡처가 작동하지 않는 이유는 무엇인가요?대부분의 바코드 캡처 문제는 키 입력 출력 및 스캔 엔진 설정의 사소한 오류에서 시작됩니다.프로필을 설정한 후 테스트 모드를 사용하여 구성을 확인해 보십시오.', metadata={'source': 'all_processed_data_ko.json', 'seq_num': 51, 'title': '바코드 캡처가 비즈니스 앱에서 작동하지 않음', 'url': 'https://docs.samsungknox.com/admin/knox-capture/kbas/barcode-capture-not-working-for-app', 'project': 'KCAP', 'last_updated': '2023-07-26', 'famil